In [1]:
# Imports and set torch device
import numpy as np
import meshplot as mp
import torch
import sys
import kaolin as kal
import matplotlib.pyplot as plt
import igl
import potpourri3d as pp3d
import trimesh

if torch.cuda.is_available():
    device = torch.device("cuda:0")
    torch.cuda.set_device(device)
else:
    device = torch.device("cpu")

print('Torch will run on:', device)

object = 'notebook'
obj_path = 'data/demo/' + object + '.obj'

Torch will run on: cuda:0


In [2]:
# Read mesh
mesh = kal.io.obj.import_mesh(
    obj_path,
    with_normals=True,
    with_materials=False,
)

vertices_tensor = mesh.vertices.to(device)
faces_tensor = mesh.faces.to(device)

vertices = vertices_tensor.detach().cpu().numpy()
faces = faces_tensor.detach().cpu().numpy()
colors = mesh.vertex_normals.cpu().numpy()

print('Number of vertices: ', vertices.shape[0])
print('Number of faces: ', faces.shape[0])

Number of vertices:  1164
Number of faces:  1574


In [3]:
# Visualize mesh
mp.plot(vertices, faces, colors)
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.10…

In [4]:
import json
colors_dict = {
    0: [1, 0, 0],   # Red
    1: [0, 1, 0],   # Green
    2: [0, 0, 1],   # Blue
    3: [1, 1, 0],   # Yellow
    4: [1, 0, 1],   # Magenta
    5: [0, 1, 1],   # Cyan
    6: [0.5, 0, 0], # Dark Red
    7: [0, 0.5, 0], # Dark Green
    8: [0, 0, 0.5], # Dark Blue
    9: [0.5, 0.5, 0.5] # Gray
}
colors_lst = ['Red', 'Green', 'Blue', 'Yellow', 'Magenta', 'Cyan', 'Dark Red', 'Dark Green', 'Dark Blue', 'Grey']
output_file = open('./outputs/demo/ABO/' + object + '/face_preds.json')
output = np.array(json.load(output_file))
segments = np.unique(output)
segments_idx = np.unique(output, return_inverse=True)[1]
prompts = segments.tolist()
if ('unknown' in prompts):
    prompts.remove('unknown')
# print('Prompts: ', prompts)
# colors = np.array([colors_dict[segment_id] for segment_id in segments_idx])
# for i in range(len(segments)):
#     print(colors_lst[i],':\t',segments[i])
# mp.plot(vertices, faces, colors, return_plot=True)
# plt.show()

In [5]:
adj = igl.adjacency_matrix(faces)
cc = igl.connected_components(adj)
print('Number of connected components:', cc[0])
# mp.plot(vertices, faces, cc[1])
# plt.show()

Number of connected components: 31


In [6]:
# mp.plot(vertices, faces, np.arange(faces.shape[0]))
# plt.show()

In [7]:
# mp.plot(vertices, faces, np.random.rand(faces.shape[0]))
# plt.show()

In [8]:
# Gaussian Curvature
k = igl.gaussian_curvature(vertices, faces)
# print('Gaussian Curvature')
# mp.plot(vertices, faces, k)
# plt.show()

In [9]:
np.random.seed(42)
i = np.random.randint(vertices.shape[0])
solver = pp3d.MeshHeatMethodDistanceSolver(
            vertices,
            faces
        )
distances = solver.compute_distance(i)
p = mp.plot(vertices, faces, distances, return_plot=True)
p.add_points(vertices[i].reshape(1, -1), shading = {'point_color':'red', 'point_size':0.2})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.10…

In [11]:
np.random.seed(42)
# i = np.random.randint(0, vertices.shape[0])
n_samples = int(vertices.shape[0] * 2)
trimeshMesh = trimesh.Trimesh(vertices, faces)
pts = trimesh.sample.sample_surface_even(trimeshMesh, n_samples)[0]
# closest_point_ind = np.argmin(np.sum((vertices[i] - pts) ** 2, axis=1))
rand_pt = np.random.randint(pts.shape[0])
pts_solver = pp3d.PointCloudHeatSolver(pts)
distances = pts_solver.compute_distance(rand_pt)
# p = mp.plot(vertices, faces, colors, return_plot=True)
p = mp.plot(pts, c=distances, shading={'point_size':0.04}, return_plot=True)
p.add_points(pts[rand_pt].reshape(1,-1), shading = {'point_color':'red', 'point_size':0.07})
plt.show()

only got 1445/2328 samples!
c:\ProgramData\miniforge3\envs\meshseg\lib\site-packages\traittypes\traittypes.py:97: UserWarning: Given trait value dtype "float32" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.725290…

In [21]:
from scipy.spatial.distance import cdist
query_vertex = np.array([[0.08,0.1,0.02]])
distances = cdist(
    pts,
    query_vertex
)
nearest_vert_ind = np.argmin(distances[:, 0])
p = mp.plot(pts, shading={'point_size':0.04, 'point_color':'blue'}, return_plot=True)
p.add_points(pts[nearest_vert_ind].reshape(1,-1), shading = {'point_color':'red', 'point_size':0.07})
p.add_points(query_vertex.reshape(1,-1), shading = {'point_color':'green', 'point_size':0.07})
plt.show()

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-3.725290…

In [ ]:
pts_solver = pp3d.PointCloudHeatSolver(vertices)
distances = pts_solver.compute_distance(i)
p = mp.plot(vertices, faces, c = distances, return_plot=True, shading = {'point_size':0.05})
p.add_points(vertices[i].reshape(1, -1), shading = {'point_color':'red', 'point_size':0.2})
plt.show()

In [ ]:
pts_solver = pp3d.PointCloudHeatSolver(vertices)
distances = pts_solver.compute_distance(i)
p = mp.plot(vertices, c = distances, return_plot=True, shading = {'point_size':0.05})
p.add_points(vertices[i].reshape(1, -1), shading = {'point_color':'red', 'point_size':0.2})
plt.show()